In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np

### 处理丢失数据
- 有两种丢失数据：
    - None
    - np.nan(NaN)

- 两种丢失数据的区别

In [2]:
type(None)

NoneType

In [4]:
type(np.nan)

float

- 在pandas中如果遇到了None形式的空值则pandas会将其强转成NAN的形式。

In [5]:
np.nan + 1

nan

In [6]:
None + 1

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

### pandas处理空值操作
- isnull
- notnull
- any
- all
- dropna
- fillna

### 面试题
- 数据说明： 
    - 数据是1个冷库的温度数据，1-7对应7个温度采集设备，1分钟采集一次。

- 数据处理目标：
    - 用1-4对应的4个必须设备，通过建立冷库的温度场关系模型，预估出5-7对应的数据。
    - 最后每个冷库中仅需放置4个设备，取代放置7个设备。
    - f(1-4) --> y(5-7)

- 数据处理过程：
    - 1、原始数据中有丢帧现象，需要做预处理；
    - 2、matplotlib 绘图；
    - 3、建立逻辑回归模型。

- 无标准答案，按个人理解操作即可，请把自己的操作过程以文字形式简单描述一下，谢谢配合。

- 测试数据为testData.xlsx


In [13]:
df = pd.read_excel('./testData.xlsx')
df

,time,none,1,2,3,4,none1,5,6,7
0,2019-01-27 17:00:00,NaN,-24.8,-18.2,-20.8,-18.8,NaN,NaN,NaN,NaN
1,2019-01-27 17:01:00,NaN,-23.5,-18.8,-20.5,-19.8,NaN,-15.2,-14.5,-16.0
2,2019-01-27 17:02:00,NaN,-23.2,-19.2,NaN,NaN,NaN,-13.0,NaN,-14.0
3,2019-01-27 17:03:00,NaN,-22.8,-19.2,-20.0,-20.5,NaN,NaN,-12.2,-9.8
4,2019-01-27 17:04:00,NaN,-23.2,-18.5,-20.0,-18.8,NaN,-10.2,-10.8,-8.8
5,2019-01-27 17:05:00,NaN,NaN,NaN,-19.0,-18.2,NaN,-10.0,-10.5,-10.8
6,2019-01-27 17:06:00,NaN,NaN,-18.5,-18.2,-17.5,NaN,NaN,NaN,NaN
7,2019-01-27 17:07:00,NaN,-24.8,-18.0,-17.5,-17.2,NaN,-14.2,-14.0,-12.5
8,2019-01-27 17:08:00,NaN,-25.2,-17.8,NaN,NaN,NaN,-16.2,NaN,-14.5
9,2019-01-27 17:09:00,NaN,-24.8,-18.2,NaN,-17.5,NaN,NaN,-15.5,-16.0


In [14]:
df.drop(labels=['none','none1'],axis=1,inplace=True)
df

,time,1,2,3,4,5,6,7
0,2019-01-27 17:00:00,-24.8,-18.2,-20.8,-18.8,NaN,NaN,NaN
1,2019-01-27 17:01:00,-23.5,-18.8,-20.5,-19.8,-15.2,-14.5,-16.0
2,2019-01-27 17:02:00,-23.2,-19.2,NaN,NaN,-13.0,NaN,-14.0
3,2019-01-27 17:03:00,-22.8,-19.2,-20.0,-20.5,NaN,-12.2,-9.8
4,2019-01-27 17:04:00,-23.2,-18.5,-20.0,-18.8,-10.2,-10.8,-8.8
5,2019-01-27 17:05:00,NaN,NaN,-19.0,-18.2,-10.0,-10.5,-10.8
6,2019-01-27 17:06:00,NaN,-18.5,-18.2,-17.5,NaN,NaN,NaN
7,2019-01-27 17:07:00,-24.8,-18.0,-17.5,-17.2,-14.2,-14.0,-12.5
8,2019-01-27 17:08:00,-25.2,-17.8,NaN,NaN,-16.2,NaN,-14.5
9,2019-01-27 17:09:00,-24.8,-18.2,NaN,-17.5,NaN,-15.5,-16.0


In [20]:
#将空值进行清洗
#1.将空对应的行数据删除

In [16]:
df.notnull().all(axis=1) #检测notnull返回的行中是否有False，如果存在false，则该行返回false

0       False
1        True
2       False
3       False
4        True
5       False
6       False
7        True
8       False
9       False
10       True
11      False
12      False
13      False
14      False
15       True
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23       True
24      False
25      False
26      False
27      False
28       True
29       True
        ...  
1030     True
1031     True
1032     True
1033     True
1034     True
1035     True
1036     True
1037     True
1038    False
1039     True
1040     True
1041     True
1042     True
1043     True
1044     True
1045     True
1046     True
1047     True
1048     True
1049     True
1050     True
1051     True
1052     True
1053     True
1054     True
1055    False
1056    False
1057    False
1058     True
1059    False
Length: 1060, dtype: bool

In [26]:
#检测哪些行中存在空值
df.isnull()


,time,1,2,3,4,5,6,7
0,False,False,False,False,False,True,True,True
1,False,False,False,False,False,False,False,False
2,False,False,False,True,True,False,True,False
3,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,False,False
5,False,True,True,False,False,False,False,False
6,False,True,False,False,False,True,True,True
7,False,False,False,False,False,False,False,False
8,False,False,False,True,True,False,True,False
9,False,False,False,True,False,True,False,False


In [27]:
#any用来检测isnull返回的行中是否存在True，如果存在true，则该行返回ture
df.isnull().any(axis=1)

0        True
1       False
2        True
3        True
4       False
5        True
6        True
7       False
8        True
9        True
10      False
11       True
12       True
13       True
14       True
15      False
16       True
17       True
18       True
19       True
20       True
21       True
22       True
23      False
24       True
25       True
26       True
27       True
28      False
29      False
        ...  
1030    False
1031    False
1032    False
1033    False
1034    False
1035    False
1036    False
1037    False
1038     True
1039    False
1040    False
1041    False
1042    False
1043    False
1044    False
1045    False
1046    False
1047    False
1048    False
1049    False
1050    False
1051    False
1052    False
1053    False
1054    False
1055     True
1056     True
1057     True
1058    False
1059     True
Length: 1060, dtype: bool

In [20]:
df.loc[df.notnull().all(axis=1)] #将空对应的行数据进行删除

,time,1,2,3,4,5,6,7
1,2019-01-27 17:01:00,-23.5,-18.8,-20.5,-19.8,-15.2,-14.5,-16.0
4,2019-01-27 17:04:00,-23.2,-18.5,-20.0,-18.8,-10.2,-10.8,-8.8
7,2019-01-27 17:07:00,-24.8,-18.0,-17.5,-17.2,-14.2,-14.0,-12.5
10,2019-01-27 17:10:00,-24.5,-18.5,-16.0,-18.5,-17.5,-16.5,-17.2
15,2019-01-27 17:15:00,-23.5,-17.8,-15.0,-18.0,10.5,10.5,10.8
23,2019-01-27 17:23:00,-23.2,-20.2,-14.0,-19.5,19.0,18.2,16.5
28,2019-01-27 17:28:00,-22.8,-15.2,-14.8,-15.2,18.8,17.2,16.2
29,2019-01-27 17:29:00,-22.5,-15.0,-14.8,-15.2,18.8,17.2,16.5
30,2019-01-27 17:30:00,-22.8,-14.8,-15.0,-15.2,18.8,17.2,16.5
32,2019-01-27 17:32:00,-22.5,-15.0,-14.8,-15.5,18.8,17.2,16.5


In [21]:
#高级用法:直接将存有缺失数据的行删除
df.dropna(axis=0)

,time,1,2,3,4,5,6,7
1,2019-01-27 17:01:00,-23.5,-18.8,-20.5,-19.8,-15.2,-14.5,-16.0
4,2019-01-27 17:04:00,-23.2,-18.5,-20.0,-18.8,-10.2,-10.8,-8.8
7,2019-01-27 17:07:00,-24.8,-18.0,-17.5,-17.2,-14.2,-14.0,-12.5
10,2019-01-27 17:10:00,-24.5,-18.5,-16.0,-18.5,-17.5,-16.5,-17.2
15,2019-01-27 17:15:00,-23.5,-17.8,-15.0,-18.0,10.5,10.5,10.8
23,2019-01-27 17:23:00,-23.2,-20.2,-14.0,-19.5,19.0,18.2,16.5
28,2019-01-27 17:28:00,-22.8,-15.2,-14.8,-15.2,18.8,17.2,16.2
29,2019-01-27 17:29:00,-22.5,-15.0,-14.8,-15.2,18.8,17.2,16.5
30,2019-01-27 17:30:00,-22.8,-14.8,-15.0,-15.2,18.8,17.2,16.5
32,2019-01-27 17:32:00,-22.5,-15.0,-14.8,-15.5,18.8,17.2,16.5


- 将空值进行填充，使用空值的近邻值进行空值的填充


In [22]:
df.fillna(value=-999) #使用指定的值填充所有的空值

,time,1,2,3,4,5,6,7
0,2019-01-27 17:00:00,-24.8,-18.2,-20.8,-18.8,-999.0,-999.0,-999.0
1,2019-01-27 17:01:00,-23.5,-18.8,-20.5,-19.8,-15.2,-14.5,-16.0
2,2019-01-27 17:02:00,-23.2,-19.2,-999.0,-999.0,-13.0,-999.0,-14.0
3,2019-01-27 17:03:00,-22.8,-19.2,-20.0,-20.5,-999.0,-12.2,-9.8
4,2019-01-27 17:04:00,-23.2,-18.5,-20.0,-18.8,-10.2,-10.8,-8.8
5,2019-01-27 17:05:00,-999.0,-999.0,-19.0,-18.2,-10.0,-10.5,-10.8
6,2019-01-27 17:06:00,-999.0,-18.5,-18.2,-17.5,-999.0,-999.0,-999.0
7,2019-01-27 17:07:00,-24.8,-18.0,-17.5,-17.2,-14.2,-14.0,-12.5
8,2019-01-27 17:08:00,-25.2,-17.8,-999.0,-999.0,-16.2,-999.0,-14.5
9,2019-01-27 17:09:00,-24.8,-18.2,-999.0,-17.5,-999.0,-15.5,-16.0


In [23]:
df.fillna(method='bfill',axis=0) #使用近邻值填充

,time,1,2,3,4,5,6,7
0,2019-01-27 17:00:00,-24.8,-18.2,-20.8,-18.8,-15.2,-14.5,-16.0
1,2019-01-27 17:01:00,-23.5,-18.8,-20.5,-19.8,-15.2,-14.5,-16.0
2,2019-01-27 17:02:00,-23.2,-19.2,-20.0,-20.5,-13.0,-12.2,-14.0
3,2019-01-27 17:03:00,-22.8,-19.2,-20.0,-20.5,-10.2,-12.2,-9.8
4,2019-01-27 17:04:00,-23.2,-18.5,-20.0,-18.8,-10.2,-10.8,-8.8
5,2019-01-27 17:05:00,-24.8,-18.5,-19.0,-18.2,-10.0,-10.5,-10.8
6,2019-01-27 17:06:00,-24.8,-18.5,-18.2,-17.5,-14.2,-14.0,-12.5
7,2019-01-27 17:07:00,-24.8,-18.0,-17.5,-17.2,-14.2,-14.0,-12.5
8,2019-01-27 17:08:00,-25.2,-17.8,-16.0,-17.5,-16.2,-15.5,-14.5
9,2019-01-27 17:09:00,-24.8,-18.2,-16.0,-17.5,-17.5,-15.5,-16.0


In [24]:
df

,time,1,2,3,4,5,6,7
0,2019-01-27 17:00:00,-24.8,-18.2,-20.8,-18.8,NaN,NaN,NaN
1,2019-01-27 17:01:00,-23.5,-18.8,-20.5,-19.8,-15.2,-14.5,-16.0
2,2019-01-27 17:02:00,-23.2,-19.2,NaN,NaN,-13.0,NaN,-14.0
3,2019-01-27 17:03:00,-22.8,-19.2,-20.0,-20.5,NaN,-12.2,-9.8
4,2019-01-27 17:04:00,-23.2,-18.5,-20.0,-18.8,-10.2,-10.8,-8.8
5,2019-01-27 17:05:00,NaN,NaN,-19.0,-18.2,-10.0,-10.5,-10.8
6,2019-01-27 17:06:00,NaN,-18.5,-18.2,-17.5,NaN,NaN,NaN
7,2019-01-27 17:07:00,-24.8,-18.0,-17.5,-17.2,-14.2,-14.0,-12.5
8,2019-01-27 17:08:00,-25.2,-17.8,NaN,NaN,-16.2,NaN,-14.5
9,2019-01-27 17:09:00,-24.8,-18.2,NaN,-17.5,NaN,-15.5,-16.0


- 作业：自定义一个携带空值的df，然后要求使用列的均值填充列中的空值

In [36]:
df = DataFrame(data=np.random.randint(0,100,size=(10,8)))
df.iloc[1,3] = None
df.iloc[3,4] = np.nan
df.iloc[5,5] = np.nan
df.iloc[2,4] = np.nan
df.iloc[2,5] = np.nan
df

,0,1,2,3,4,5,6,7
0,79,8,36,66.0,22.0,28.0,84,26
1,2,5,66,NaN,40.0,27.0,49,97
2,75,84,90,38.0,NaN,NaN,30,7
3,2,86,50,95.0,NaN,68.0,39,28
4,48,95,82,55.0,52.0,65.0,66,35
5,66,34,45,19.0,66.0,NaN,12,73
6,77,47,27,98.0,47.0,29.0,76,17
7,97,57,37,25.0,93.0,74.0,28,96
8,94,57,58,1.0,74.0,74.0,19,70
9,63,45,84,33.0,65.0,74.0,29,62


### 面试题
- 数据说明： 
    - 数据是1个冷库的温度数据，1-7对应7个温度采集设备，1分钟采集一次。

- 数据处理目标：
    - 用1-4对应的4个必须设备，通过建立冷库的温度场关系模型，预估出5-7对应的数据。
    - 最后每个冷库中仅需放置4个设备，取代放置7个设备。
    - f(1-4) --> y(5-7)

- 数据处理过程：
    - 1、原始数据中有丢帧现象，需要做预处理；
    - 2、matplotlib 绘图；
    - 3、建立逻辑回归模型。

- 无标准答案，按个人理解操作即可，请把自己的操作过程以文字形式简单描述一下，谢谢配合。

- 测试数据为testData.xlsx


### 处理重复数据

### 处理异常数据
- 自定义一个1000行3列（A，B，C）取值范围为0-1的数据源，然后将C列中的值大于其两倍标准差的异常值进行清洗